In [1]:
import pandas as pd

# Upload the dataset using Colab's built-in function
from google.colab import files
uploaded = files.upload()


Saving credit_card_default.csv to credit_card_default.csv


In [2]:
train_df = pd.read_csv('credit_card_default.csv')


In [3]:
train_df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

# Imputation
numerical_cols = [col for col in train_df.columns if (train_df[col].dtype == 'float64' or train_df[col].dtype == 'int64') and col != 'default payment next month']
skewness = train_df[numerical_cols].skew()

# Impute numerical columns
for col in numerical_cols:
    if skewness[col] < -0.5 or skewness[col] > 0.5:
        train_df[col].fillna(train_df[col].median(), inplace=True)
    else:
        train_df[col].fillna(train_df[col].mean(), inplace=True)

categorical_cols = [col for col in train_df.columns if train_df[col].dtype == 'object' and col != 'default payment next month']

# Impute categorical columns
for col in categorical_cols:
    train_df[col].fillna(train_df[col].mode()[0], inplace=True)

# Outlier Handling
def cap_outliers_using_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[column] = df[column].apply(lambda x: upper_bound if x > upper_bound else x)
    df[column] = df[column].apply(lambda x: lower_bound if x < lower_bound else x)
    return df

for col in numerical_cols:
    train_df = cap_outliers_using_iqr(train_df, col)

# Encoding
label_encoders = {}
one_hot_encoders = {}

label_encode_cols = [col for col in train_df.columns if train_df[col].dtype == 'object' and train_df[col].nunique() <= 3]
one_hot_encode_cols = [col for col in train_df.columns if train_df[col].dtype == 'object' and train_df[col].nunique() > 3]

for col in label_encode_cols:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])
    label_encoders[col] = le

ohe = OneHotEncoder(drop='first', sparse=False)
if one_hot_encode_cols:
    train_encoded = ohe.fit_transform(train_df[one_hot_encode_cols])
    train_encoded_df = pd.DataFrame(train_encoded, columns=ohe.get_feature_names_out(one_hot_encode_cols))

    train_df.drop(one_hot_encode_cols, axis=1, inplace=True)
    train_df = pd.concat([train_df, train_encoded_df], axis=1)

    one_hot_encoders = ohe

# Scaling
scaler = MinMaxScaler()
feature_columns = [col for col in train_df.columns if col != 'default payment next month']

train_df_scaled = train_df.copy()
train_df_scaled[feature_columns] = scaler.fit_transform(train_df[feature_columns])
train_df_scaled.head()


,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,0.000000,0.019417,1.0,0.571429,0.0,0.075949,1.000000,1.000000,0.285714,0.285714,...,0.363852,0.365899,0.368450,0.000000,0.061242,0.000000,0.000000,0.000000,0.000000,1
1,0.000033,0.213592,1.0,0.571429,0.5,0.126582,0.285714,1.000000,0.571429,0.571429,...,0.379529,0.383735,0.385455,0.000000,0.088885,0.093655,0.104285,0.000000,0.203596,1
2,0.000067,0.155340,1.0,0.571429,0.5,0.329114,0.571429,0.571429,0.571429,0.571429,...,0.432514,0.443066,0.449532,0.137812,0.133327,0.093655,0.104285,0.103093,0.508990,0
3,0.000100,0.077670,1.0,0.571429,0.0,0.405063,0.571429,0.571429,0.571429,0.571429,...,0.499510,0.515395,0.522526,0.181571,0.179459,0.112386,0.114713,0.110206,0.101798,0
4,0.000133,0.077670,0.0,0.571429,0.0,0.911392,0.285714,0.571429,0.285714,0.571429,...,0.464179,0.464737,0.468211,0.181571,1.000000,0.936549,0.938563,0.071031,0.069121,0


In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier
from xgboost import XGBClassifier

# ... [Your preprocessing code here] ...

# Splitting the dataset
X = train_df_scaled.drop('default payment next month', axis=1)
y = train_df_scaled['default payment next month']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Initialize model info list
model_info = []

# Define hyperparameter tuning function
def hyperparameter_tuning(model, param_grid, X_train, y_train):
    skfold = StratifiedKFold(n_splits=5)
    random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_jobs=-1, n_iter=50, cv=skfold, verbose=1, scoring="accuracy")
    random_result = random_search.fit(X_train, y_train)
    return random_result.best_score_, random_result.best_params_

# kNN
params_knn = {
    'n_neighbors': list(range(1, 10)),
    'weights': ['uniform'],
    'algorithm': ['auto']
}
best_score_knn, best_params_knn = hyperparameter_tuning(KNeighborsClassifier(), params_knn, X_train, y_train)

# Results
results = {
    'kNN': (best_score_knn, best_params_knn),
    # 'SVM': (best_score_svm, best_params_svm),
    # 'Gradient Boosting': (best_score_gb, best_params_gb),
    # 'AdaBoost': (best_score_ada, best_params_ada),
    # 'XGBoost': (best_score_xgb, best_params_xgb)
}

for model, (score, params) in results.items():
    print(f"Model: {model}")
    print(f"Best Score (accuracy): {score}")
    print(f"Best Parameters: {params}")
    print("-"*60)


Fitting 5 folds for each of 9 candidates, totalling 45 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 9 is smaller than n_iter=50. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Model: kNN
Best Score (accuracy): 0.7987083333333332
Best Parameters: {'weights': 'uniform', 'n_neighbors': 9, 'algorithm': 'auto'}
------------------------------------------------------------


In [11]:

# SVM
params_svm = {
    'C': [0.1, 1, 10],
    'kernel': ['linear'],
    'gamma': ['auto']
}
best_score_svm, best_params_svm = hyperparameter_tuning(SVC(), params_svm, X_train, y_train)

# Gradient Boosting
params_gb = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': list(range(3, 10)),
    'subsample': [0.5, 1.0],
    'max_features': ['auto']
}
best_score_gb, best_params_gb = hyperparameter_tuning(GradientBoostingClassifier(), params_gb, X_train, y_train)

# AdaBoost
params_ada = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 1.0]
}
best_score_ada, best_params_ada = hyperparameter_tuning(AdaBoostClassifier(), params_ada, X_train, y_train)

# XGBoost
params_xgb = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1],
    'max_depth': list(range(3, 15)),
    'subsample': [0.5,1.0],
    'colsample_bytree': [0.5, 1.0]
}
best_score_xgb, best_params_xgb = hyperparameter_tuning(XGBClassifier(), params_xgb, X_train, y_train)


Fitting 5 folds for each of 3 candidates, totalling 15 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 3 is smaller than n_iter=50. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 50 candidates, totalling 250 fits


/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
/usr/local/lib/python3.10/di

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Model: kNN
Best Score (accuracy): 0.7987083333333332
Best Parameters: {'weights': 'uniform', 'n_neighbors': 9, 'algorithm': 'auto'}
------------------------------------------------------------


In [12]:
# Results
results = {
    'kNN': (best_score_knn, best_params_knn),
    'SVM': (best_score_svm, best_params_svm),
    'Gradient Boosting': (best_score_gb, best_params_gb),
    'AdaBoost': (best_score_ada, best_params_ada),
    'XGBoost': (best_score_xgb, best_params_xgb)
}

for model, (score, params) in results.items():
    print(f"Model: {model}")
    print(f"Best Score (accuracy): {score}")
    print(f"Best Parameters: {params}")
    print("-"*60)


Model: kNN
Best Score (accuracy): 0.7987083333333332
Best Parameters: {'weights': 'uniform', 'n_neighbors': 9, 'algorithm': 'auto'}
------------------------------------------------------------
Model: SVM
Best Score (accuracy): 0.7787916666666667
Best Parameters: {'kernel': 'linear', 'gamma': 'auto', 'C': 0.1}
------------------------------------------------------------
Model: Gradient Boosting
Best Score (accuracy): 0.8230000000000001
Best Parameters: {'subsample': 0.5, 'n_estimators': 150, 'max_features': 'auto', 'max_depth': 3, 'learning_rate': 0.05}
------------------------------------------------------------
Model: AdaBoost
Best Score (accuracy): 0.8197083333333334
Best Parameters: {'n_estimators': 50, 'learning_rate': 0.01}
------------------------------------------------------------
Model: XGBoost
Best Score (accuracy): 0.822625
Best Parameters: {'subsample': 0.5, 'n_estimators': 150, 'max_depth': 4, 'learning_rate': 0.01, 'colsample_bytree': 1.0}
--------------------------------

In [13]:
# Assuming 'results' is the dictionary containing models' results

# Identify the best model and its parameters
best_model_name, (best_score, best_params) = max(results.items(), key=lambda x: x[1][0])

print(f"Best Model: {best_model_name}")
print(f"Best Score (accuracy): {best_score}")
print(f"Best Parameters: {best_params}")

# Instantiate the best model with the best parameters
if best_model_name == 'kNN':
    best_model_instance = KNeighborsClassifier(**best_params)
elif best_model_name == 'SVM':
    best_model_instance = SVC(**best_params)
elif best_model_name == 'Gradient Boosting':
    best_model_instance = GradientBoostingClassifier(**best_params)
elif best_model_name == 'AdaBoost':
    best_model_instance = AdaBoostClassifier(**best_params)
elif best_model_name == 'XGBoost':
    best_model_instance = XGBClassifier(**best_params)

# Fit the model on the training data
best_model_instance.fit(X_train, y_train)

# Use the model to make predictions on test data
y_pred = best_model_instance.predict(X_test)

# If you want to evaluate the performance, you can use accuracy or other metrics:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the best model on test data: {accuracy}")


Best Model: Gradient Boosting
Best Score (accuracy): 0.8230000000000001
Best Parameters: {'subsample': 0.5, 'n_estimators': 150, 'max_features': 'auto', 'max_depth': 3, 'learning_rate': 0.05}


/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
/usr/local/lib/python3.10/di

Accuracy of the best model on test data: 0.8178333333333333


/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(


In [14]:
print(f"Accuracy of the best model on test data: {accuracy}")



Accuracy of the best model on test data: 0.8178333333333333
